# Yield Term Structure and Discounting Swap Engine in QuantLib

This notebook demonstrates how to set up a flat yield term structure and a swap engine using QuantLib. We create a `YieldTermStructureHandle` with a constant rate and use it within a `DiscountingSwapEngine` to calculate the net present value (NPV) of a vanilla swap.

In [1]:

# Import the QuantLib library
import QuantLib as ql


## Step 1: Create the Yield Term Structure

In [2]:

# Setting up a flat yield curve with a constant rate of 0.5%
# '2' is the settlement days, meaning the yield curve will be effective in two days
# ql.TARGET() represents the calendar (TARGET is the Trans-European Automated Real-time Gross Settlement Express Transfer System)
# 0.5 is the interest rate (in decimal format)
# ql.Actual360() is the day count convention

yield_curve = ql.FlatForward(2, ql.TARGET(), 0.5, ql.Actual360())
yts = ql.YieldTermStructureHandle(yield_curve)

# Display the rate to understand how it's set
print("Interest rate set for the yield curve:", yield_curve.forwardRate(ql.Date(2, 1, 2025), ql.Date(2, 1, 2025), ql.Actual360(), ql.Simple).rate())


Interest rate set for the yield curve: 0.5000125002085909


## Step 2: Set Up the Discounting Engine

In [3]:

# Set up the DiscountingSwapEngine using the yield term structure created above
engine = ql.DiscountingSwapEngine(yts)

# Output to show that the engine has been initialized with the yield term structure
print("Discounting Swap Engine created with the yield curve.")


Discounting Swap Engine created with the yield curve.


## Step 3: Test the Engine with a Basic Swap

In [5]:

# Set up basic parameters for a vanilla swap
notional = 1000000  # 1 million currency units
fixed_rate = 0.05   # 5% fixed rate
floating_spread = 0.0025  # 25 basis points floating spread

# Define the swap schedule
start_date = ql.Date(2, 1, 2025)
maturity_date = ql.Date(2, 1, 2030)
tenor = ql.Period(ql.Semiannual)

# Fixed and floating legs
fixed_schedule = ql.Schedule(start_date, maturity_date, tenor, ql.TARGET(), ql.ModifiedFollowing, ql.ModifiedFollowing, ql.DateGeneration.Forward, False)
floating_schedule = fixed_schedule

# Fixed leg
fixed_leg = ql.FixedRateLeg(fixed_schedule, ql.Actual360(), [notional], [fixed_rate])

# Floating leg setup with a spread, without using withSpreads
index = ql.Euribor6M(yts)  # Define the floating index
floating_leg = ql.IborLeg([notional], floating_schedule, index, ql.Actual360())

# Manually add the spread to the floating cash flows
floating_leg_cashflows = [cf.amount() + (floating_spread * notional * cf.date().serialNumber()) for cf in floating_leg]

# Create the swap
# Create the vanilla swap directly using schedules and parameters
swap = ql.VanillaSwap(
    ql.VanillaSwap.Payer,
    notional,
    fixed_schedule,  # fixed leg schedule
    fixed_rate,      # fixed leg rate
    ql.Actual360(),  # fixed leg day count convention
    floating_schedule,  # floating leg schedule
    index,           # floating index
    floating_spread, # floating spread
    ql.Actual360()   # floating leg day count convention
)


# Set the engine to the swap
swap.setPricingEngine(engine)

# Calculate and display the NPV of the swap
npv = swap.NPV()
print("Swap NPV:", npv)


Swap NPV: 772183.1345742819


## Explanation of Key Components

1. **`FlatForward`**: Creates a flat (constant) yield curve, assuming a single interest rate across all maturities.
2. **`YieldTermStructureHandle`**: Provides flexible access to the yield curve, allowing dynamic updates.
3. **`DiscountingSwapEngine`**: Utilizes the yield curve to discount cash flows, essential for pricing swaps.
4. **Vanilla Swap**: In this example, the swap has both a fixed and floating leg, with cash flows discounted using the `DiscountingSwapEngine`.